In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'f:\\NamHoang\\MyProject\\DL\\dog-classification'

In [4]:
from pathlib import Path
from dataclasses import dataclass

In [5]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.utils.common import read_yaml, create_dir
from cnnClassifier.constants import *

In [7]:
class ConfigurationManager:
    def __init__(self) -> None:
        config_path = CONFIG_PATH
        params_path = PARAMS_PATH
        
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_dir([self.config['artifacts_root']])
        
    def get_data_ingestion_config(self):
        config = self.config['data_ingestion']
        
        create_dir([config['root_dir']])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config['root_dir'],
            source_url=config['source_url'],
            local_data_file=config['local_data_file'],
            unzip_dir=config['unzip_dir']
        )
        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from cnnClassifier import logger

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config
    
    def download_data(self):
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='

            gdown.download(prefix +file_id, zip_download_dir)
            logger.info(f"Download data from {dataset_url} into file {zip_download_dir} successfully")
        except Exception as e:
            logger.error(f"Unexpected error: {e}")
            raise e
    
    def extract_zip_file(self):
        zip_download_dir = self.config.local_data_file
        unzip_dir = self.config.unzip_dir
        try:
            os.makedirs(unzip_dir, exist_ok=True)
            with zipfile.ZipFile(zip_download_dir, 'r') as zip_ref:
                zip_ref.extractall(unzip_dir)
            logger.info(f"Successfully extracted {zip_download_dir} to {unzip_dir}")
        except Exception as e:
            logger.error(f"Error: {e} during zip file extraction")
            raise e

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except:
    raise

[2024-01-12 21:46:15,569: INFO: common: Yaml file: config\config.yaml loaded successfully]
[2024-01-12 21:46:15,571: INFO: common: Yaml file: params.yaml loaded successfully]
[2024-01-12 21:46:15,572: INFO: common: Created directory at: artifacts]
[2024-01-12 21:46:15,572: INFO: common: Created directory at: artifacts/data_ingestion]
